<a href="https://colab.research.google.com/github/Kenichi-Iwase/TestPrograms/blob/master/DeepLearningZero3_Ver01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VariableクラスとFunctionクラスの定義
出典　ゼロからつくるディープラーニング3

In [1]:
import numpy as np

## Variableクラスを定義
self.dataはdataオブジェクトを格納。self.grad は一旦 None で定義。

In [2]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None

実装

In [3]:
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


# Functionクラスの定義
## forwardメソッド実装
Pythonではクラスだが、Javaではインタフェースと呼ばれている。本書ではポリモーフィズムまたはストラテジパターンを使用してFunctionクラスの実装を見直している。
*   Functionクラスは基底クラスとする
*   具体的な関数はFunctionクラスを継承したクラスで実装する

## backwardメソッドを実装
6章でbackwardメソッドを実装

In [4]:
class Function:
  def __call__(self,input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input  # 入力された変数を覚える
    return output

  def forward(self,x):
    raise NotImplementedError()

  def backward(self,gy):
    raise NotImplementedError()

# Squareクラスの定義
## forwardメソッドの実装
forwardメソッドの実装
## backwardメソッドの実装
6章でbackwardメソッドの実装。self.input.data とは何者なのか。微分した結果を入れるのがポイント。

In [5]:
class Square(Function):
  def forward(self,x):
    return x ** 2

  def backward(self,gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

# VariableとFunctionオブジェクトの作成
type()を使って、オブジェクトの型を取得することができる。
f = Square()の箇所を f = Function()と記述するとNotImplementedErrorの例外が発生する。これは基底クラスのforward()メソッドが内部でコールされたため。

In [6]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# Expクラスの定義
## forwardメソッド
## backwardメソッド
self.input.data とは何者なのか。微分した結果を入れるのがポイント。

In [7]:
class Exp(Function):
  def forward(self,x):
    return np.exp(x)

  def backward(self,gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

# Expクラスの動作検証
## 関数を連結して実行

In [8]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## 検証
MATLABでは exp(0.5.^2).^2、Pythonは下記。

In [9]:
print(np.exp(0.5**2)**2)

1.648721270700128


# 中心差分近似を用いて数値微分を求める関数の定義

In [10]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

# 動作検証
## Squareクラス

In [11]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f,x)
print(dy)

4.000000000004


In [12]:
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)
print(dy)

3.2974426293330694


#バックプロパゲーションの実装
6章のバックプロパゲーションを実装

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
c = C(b)

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256
